In [1]:
# This document is for making 3D plots of MHD-AEPIC runs, using both PIC and MHD data. 
# The PIC data and MHD data can be read in at two different cadences. The PIC data overrides the MHD data

# Essential Jupyter Notebook Magic
%matplotlib inline

# General Purpose and Data Handling Libraries
import os
import re
import glob
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from natsort import natsorted
import pickle
from operator import add
import random

# MatPlotlib for Plotting and Visualization
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.tri as tri
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib import cm, ticker
from matplotlib.colors import LogNorm, LightSource, ListedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import LogFormatter, LogFormatterSciNotation
from matplotlib.ticker import LogLocator, MultipleLocator, NullFormatter
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d import Axes3D
from streamtracer import StreamTracer, VectorGrid
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from cmap import Colormap
from matplotlib.ticker import MaxNLocator


# Scipy for Scientific Computing and Analysis
from scipy import stats, interpolate
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d, griddata
from scipy.ndimage import label, gaussian_filter
from scipy.spatial import ConvexHull
from scipy.interpolate import RegularGridInterpolator
from skimage import measure
from shapely.geometry import Polygon

# Image Handling and Processing
from PIL import Image

# Tecplot for Scientific Data Visualization
import tecplot as tp
from tecplot.exception import *
from tecplot.constant import *

# For 3d plotting
from skimage import measure


In [143]:
# Define Constants
amu = 1.67e-27
k_b = 1.38e-23
mu_0 = 1.257e-6
R_M = 2440e3 #m
m_p = 1.67e-27 # kg
e = 1.60218e-19 # C

# Define utility functions
def read_dataset(mypath,port=7600):
    # Reads in file "mypath" and returns a dataset object. May take a while for larger files.

    print("reading:",mypath)
    # First connect to TecPlot
    tp.session.connect(port=port)

    # Configure layout
    tp.new_layout()
    dataset = tp.data.load_tecplot(mypath)
    frame = tp.active_frame()
    frame.plot_type = PlotType.Cartesian3D

    # Return dataset
    return dataset

def Bz_dip(x_array,y_array,z_array):
    # Input: arrays of x,y,z (in planet centered coords).
    # Output: Bz at each point
    
    return - 200.9 * (3*(z_array-0.2)**2 - (x_array**2+y_array**2+(z_array-0.2)**2))/((x_array**2+y_array**2+(z_array-0.2)**2)**(5/2))

def get_files(dir, start_time, t_bound, dt, key=".*cut_particle_region0_0.*", read_time = False, reduce = True):
    # For a directory "dir", return a list of all files which match the regex expression "key"
    
    all_files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files=[]
    for file in all_files:
        match = re.search(key,file)
        if match != None:
            files.append(file)
    files.sort()
    
    # Now give them the appropriate name for their time
    # If we haven't already named these files with their time, do that now
    named_files = {}
    if read_time == False:
        for i in range(len(files)):
            time = round(i*dt+start_time,3)
            named_files[time] = files[i]
    # Otherwise, read the time right from the (last 6 elements) filename
    else:
        for i in range(len(files)):
            time = str("%.2f"%float(files[i][-6:]))
            named_files[time] = files[i]
    
    # Now cut the list down to files inside t_bound
    if reduce:
        reduced_files = {}
        for file_time in list(named_files.keys()):#[int((t_bound[0]-start_time)/dt):int((t_bound[1]-start_time)/dt)]: #only loop over the times within t_bound
            if t_bound[0]<=float(file_time)<t_bound[1]:
                reduced_files[file_time] = str(named_files[file_time])
        return reduced_files

    else:
        return named_files

def dat_to_plt(dir,files):
    # Hand it a directory with the dict of files in it, and it will convert them to .plt and save in dir
    
    for file in files:
        dataset=read_dataset(str(dir+files[file]))
        print("saving file:",str(dir+files[file][:-3]+"plt"))
        tp.data.save_tecplot_plt(str(dir+files[file][:-3]+"plt"))
        os.remove(str(dir+files[file]))
        print(f"Deleted original .dat file: {files[file]}")

def plt_to_numpy(dataset,var_ls=["Bz"],save_cs = True):
    # Input: the path to a .plt file, and a list of variables to convert into a numpy meshgrid
    # Output: a dictionary of arrays, each labelled according to its name in var_ls
    # Var_ls should be *extensive*, so that this long process does not need to be rerun

    # Extract the coordinate axes
    x_axis = np.unique(dataset.variable("X").values(0).as_numpy_array())
    y_axis = np.unique(dataset.variable("Y").values(0).as_numpy_array())[1:-1]
    z_axis = np.unique(dataset.variable("Z").values(0).as_numpy_array())

    # Create an ordered zone
    rect_zone = dataset.add_ordered_zone('rect_zone',[len(x_axis),len(y_axis-2),len(z_axis)])

    # Create 3D coordinate meshgrids
    xxx,yyy,zzz = np.meshgrid(x_axis,y_axis,z_axis)

    # Assign coordinate values to the rect_zone using the meshgrids
    rect_zone.values('X')[:] = xxx.ravel()
    rect_zone.values('Y')[:] = yyy.ravel()
    rect_zone.values('Z')[:] = zzz.ravel()

    # Compute derivatives in tecplot, which does it efficiently
    # Compute current density, in A/m^2
    if ("Jx" in var_ls) or ("Jy" in var_ls) or ("Jz" in var_ls):
        print("Computing J = ∇xB")
        tp.data.operate.execute_equation(equation='{Jx} = (ddy({Bz}) - ddz({By}))/(1.2566*10**(-6))/2440000*10**(-9)',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{Jy} = (ddz({Bx}) - ddx({Bz}))/(1.2566*10**(-6))/2440000*10**(-9)',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{Jz} = (ddx({By}) - ddy({Bx}))/(1.2566*10**(-6))/2440000*10**(-9)',
            ignore_divide_by_zero=True)

    # Compute plasma pressure gradient, in nPa / m
    if ("dp_dx" in var_ls) or ("dp_dy" in var_ls) or ("dp_dz" in var_ls):
        print("Computing ∇$p$")
        tp.data.operate.execute_equation(equation='{dp_dx} = (ddx({pxxS1}+{pxxS0}+{pyyS1}+{pyyS0}+{pzzS1}+{pzzS0}))/3/2440000',
        ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dp_dy} = (ddy({pxxS1}+{pxxS0}+{pyyS1}+{pyyS0}+{pzzS1}+{pzzS0}))/3/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dp_dz} = (ddz({pxxS1}+{pxxS0}+{pyyS1}+{pyyS0}+{pzzS1}+{pzzS0}))/3/2440000',
            ignore_divide_by_zero=True)
        
    # Compute magnetic field gradient , in nT / m
    if ("dB_dx" in var_ls) or ("dB_dy" in var_ls) or ("dB_dz" in var_ls): 
        print("Computing ∇B")
        tp.data.operate.execute_equation(equation='{dB_dx} = (ddx(({Bx}*{Bx}+{By}*{By}+{Bz}*{Bz})**(0.5)))/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dB_dy} = (ddy(({Bx}*{Bx}+{By}*{By}+{Bz}*{Bz})**(0.5)))/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dB_dz} = (ddz(({Bx}*{Bx}+{By}*{By}+{Bz}*{Bz})**(0.5)))/2440000',
            ignore_divide_by_zero=True)

    # Compute advective derivative (u . del) u
    if ("duix_dx" in var_ls):
        print("Computing jacobian for $(u_i\cdot∇)u_i$")
        tp.data.operate.execute_equation(equation='{duix_dx} = (ddx({uxS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiy_dx} = (ddx({uyS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiz_dx} = (ddx({uzS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duix_dy} = (ddy({uxS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiy_dy} = (ddy({uyS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiz_dy} = (ddy({uzS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duix_dz} = (ddz({uxS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiy_dz} = (ddz({uyS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duiz_dz} = (ddz({uzS1}))/2440000', # km/s /m
            ignore_divide_by_zero=True)

    print("Beginning interpolation...")
    # Interpolate onto rect_zone
    tp.data.operate.interpolate_linear(source_zones=[0],
        destination_zone=1,
        fill_value=0)

    # Define dictionary to save results
    data3d = {"X":xxx,"Y":yyy,"Z":zzz}

    # All all variables to data
    for var in var_ls:
        data3d[var] = rect_zone.values(var).as_numpy_array().reshape(xxx.shape)

    # Save in place
    print("Extraction complete! Saving 3D data ...")
    save_file = open(str(dir+file[:-4]+"_numpy_t_"+'{:06.2f}'.format(round(time,2))), 'wb') 
    pickle.dump(data3d, save_file) 
    print("Done!")

    if save_cs:
        #Calculate the plasma beta meshgrid
        beta_meshgrid = (2*mu_0*(rect_zone.values("pxxS0").as_numpy_array()+rect_zone.values("pyyS0").as_numpy_array()+rect_zone.values("pzzS0").as_numpy_array()+rect_zone.values("pxxS1").as_numpy_array()+
                                rect_zone.values("pyyS1").as_numpy_array()+rect_zone.values("pzzS1").as_numpy_array())*1e9/3/(rect_zone.values("Bx").as_numpy_array()**2+rect_zone.values("By").as_numpy_array()**2+rect_zone.values("Bz").as_numpy_array()**2)).reshape(xxx.shape)
        beta_meshgrid[np.isnan(beta_meshgrid)] = -1
        
         # New code: extract all of the Z coords, smooth them, and then find the values interpolated to those points!
        data = {"X":xxx[:,:,0],"Y":yyy[:,:,0]} #, "Z":np.zeros_like(xxx[:,:,0])}
        print("Saving cs data...")
        # Define empty array to save the unsmoothed Z values to
        Z_rough = np.zeros_like(xxx[:,:,0])+0.2
        # At each x/y, find the z coord of max beta and save that
        for idy in range(len(yyy[:,0,0])):
            for idx in range(len(xxx[0,:,0])):
                # If any of the beta values in this column are -1 near the middle, that means we are at the inner edge of the bounding box
                # In this case, set the data value to 0 to let me know where the boundary is!
                if np.min(beta_meshgrid[idy,idx,len(z_axis)//4:-len(z_axis)//4]) == -1:
                    Z_rough[idy,idx] = 0.2
                else:
                    idz = np.argmax(beta_meshgrid[idy,idx,:])
                    Z_rough[idy,idx] = zzz[idy,idx,idz]
    
        # Smoothing parameter
        smoothing_param = 5
        # Smooth the Z meshgrid
        data['Z'] = smooth_meshgrid(xxx[:,:,0], yyy[:,:,0], Z_rough, smoothing_param)
    
        # Use this as a template to extract all the other data with
        for name in var_ls:
            data[name] = np.zeros_like(xxx[:,:,0])
    
            # Extract each variable from tecplot as an array
            var = rect_zone.values(name).as_numpy_array().reshape(xxx.shape)
    
            # At each x/y, find the z coord of max beta and save that
            for idy in range(len(yyy[:,0,0])):
                for idx in range(len(xxx[0,:,0])):
                    # If any of the beta values in this column are -1 near the middle, that means we are at the inner edge of the bounding box
                    # In this case, set the data value to 0 to let me know where the boundary is!
                    if np.min(beta_meshgrid[idy,idx,len(z_axis)//4:-len(z_axis)//4]) == -1:
                        data[name][idy,idx] = 0
                    else:
                        # Find the indices of the two nearest points
                        lower_idz = np.searchsorted(zzz[idy,idx,:], data['Z'][idy,idx]) - 1
                        upper_idz = lower_idz + 1
    
                        # Get the coordinates of the nearest points
                        Z_lower = zzz[idy,idx,lower_idz]
                        Z_upper = zzz[idy,idx,upper_idz]
                        var_lower = var[idy,idx,lower_idz]
                        var_upper = var[idy,idx,upper_idz]
                        
                        # Perform linear interpolation
                        data[name][idy,idx] = var_lower + (var_upper - var_lower) * (data['Z'][idy,idx] - Z_lower) / (Z_upper - Z_lower)
    
        print("Done!")
        save_file = open(str(dir+file[:-4]+"_csdata_t_"+'{:06.2f}'.format(round(time,2))), 'wb') 
        pickle.dump(data, save_file) 

def MHD_to_numpy(dataset,x_axis,y_axis,z_axis,var_ls=["Bz"],save_cs = True):
    # Input: the path to a .plt file, and a list of variables to convert into a numpy meshgrid
    # Output: a dictionary of arrays, each labelled according to its name in var_ls
    # Var_ls should be *extensive*, so that this long process does not need to be rerun

    # Create an ordered zone`
    rect_zone = dataset.add_ordered_zone('rect_zone',[len(x_axis),len(y_axis),len(z_axis)])

    # Create 3D coordinate meshgrids
    xxx,yyy,zzz = np.meshgrid(x_axis,y_axis,z_axis)

    # Assign coordinate values to the rect_zone using the meshgrids
    rect_zone.values('X [[]R[]]')[:] = xxx.ravel()
    rect_zone.values('Y [[]R[]]')[:] = yyy.ravel()
    rect_zone.values('Z [[]R[]]')[:] = zzz.ravel()

    # Compute derivatives in tecplot, which does it efficiently
    # Compute plasma pressure gradient, in nPa / m
    if ("dp_dx" in var_ls) or ("dp_dy" in var_ls) or ("dp_dz" in var_ls):
        print("Computing ∇$p$")
        tp.data.operate.execute_equation(equation='{dp_dx} = (ddx({P [[]nPa[]]}))/3/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dp_dy} = (ddy({P [[]nPa[]]}))/3/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dp_dz} = (ddz({P [[]nPa[]]}))/3/2440000',
            ignore_divide_by_zero=True)
        
    # Compute magnetic field gradient , in nT / m
    if ("dB_dx" in var_ls) or ("dB_dy" in var_ls) or ("dB_dz" in var_ls): 
        print("Computing ∇B")
        tp.data.operate.execute_equation(equation='{dB_dx} = (ddx(({B_x [[]nT[]]}*{B_x [[]nT[]]}+{B_y [[]nT[]]}*{B_y [[]nT[]]}+{B_z [[]nT[]]}*{B_z [[]nT[]]})**(0.5)))/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dB_dy} = (ddy(({B_x [[]nT[]]}*{B_x [[]nT[]]}+{B_y [[]nT[]]}*{B_y [[]nT[]]}+{B_z [[]nT[]]}*{B_z [[]nT[]]})**(0.5)))/2440000',
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dB_dz} = (ddz(({B_x [[]nT[]]}*{B_x [[]nT[]]}+{B_y [[]nT[]]}*{B_y [[]nT[]]}+{B_z [[]nT[]]}*{B_z [[]nT[]]})**(0.5)))/2440000',
            ignore_divide_by_zero=True)

    # Compute advective derivative (u . del) u
    if ("duix_dx" in var_ls):
        print("Computing jacobian for $(u_i\cdot∇)u_i$")
        tp.data.operate.execute_equation(equation='{dux_dx} = (ddx({U_x [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duy_dx} = (ddx({U_y [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duz_dx} = (ddx({U_z [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dux_dy} = (ddy({U_x [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duy_dy} = (ddy({U_y [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duz_dy} = (ddy({U_z [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{dux_dz} = (ddz({U_x [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duy_dz} = (ddz({U_y [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)
        tp.data.operate.execute_equation(equation='{duz_dz} = (ddz({U_z [km/s]}))/2440000', # km/s /m
            ignore_divide_by_zero=True)

    print("Beginning interpolation...")
    # Interpolate onto rect_zone
    tp.data.operate.interpolate_linear(source_zones=[0],
        destination_zone=1,
        fill_value=0)

    # Define dictionary to save results
    data3d = {"X":xxx,"Y":yyy,"Z":zzz}

    # All all variables to data
    for var in var_ls:
        data3d[var] = rect_zone.values(var).as_numpy_array().reshape(xxx.shape)

    # Save in place
    print("Extraction complete! Saving 3D data ...")
    save_file = open(str(dir+file[:-4]+"MHDTEST_numpy_t_"+'{:06.2f}'.format(round(time,2))), 'wb') 
    pickle.dump(data3d, save_file) 
    print("Done!")

    if save_cs:
        #Calculate the plasma beta meshgrid
        beta_meshgrid = (2*mu_0*rect_zone.values("P [[]nPa[]]").as_numpy_array()*1e9/(rect_zone.values("B_x [[]nT[]]").as_numpy_array()**2+rect_zone.values("B_y [[]nT[]]").as_numpy_array()**2+rect_zone.values("B_z [[]nT[]]").as_numpy_array()**2)).reshape(xxx.shape)
        beta_meshgrid[np.isnan(beta_meshgrid)] = -1
        
         # New code: extract all of the Z coords, smooth them, and then find the values interpolated to those points!
        data = {"X":xxx[:,:,0],"Y":yyy[:,:,0]} #, "Z":np.zeros_like(xxx[:,:,0])}
        print("Saving cs data...")
        # Define empty array to save the unsmoothed Z values to
        Z_rough = np.zeros_like(xxx[:,:,0])+0.2
        # At each x/y, find the z coord of max beta and save that
        for idy in range(len(yyy[:,0,0])):
            for idx in range(len(xxx[0,:,0])):
                # If any of the beta values in this column are -1 near the middle, that means we are at the inner edge of the bounding box
                # In this case, set the data value to 0 to let me know where the boundary is!
                if np.min(beta_meshgrid[idy,idx,len(z_axis)//4:-len(z_axis)//4]) == -1:
                    Z_rough[idy,idx] = 0.2
                else:
                    idz = np.argmax(beta_meshgrid[idy,idx,:])
                    Z_rough[idy,idx] = zzz[idy,idx,idz]
    
        # Smoothing parameter
        smoothing_param = 5
        # Smooth the Z meshgrid
        data['Z'] = smooth_meshgrid(xxx[:,:,0], yyy[:,:,0], Z_rough, smoothing_param)
    
        # Use this as a template to extract all the other data with
        for name in var_ls:
            data[name] = np.zeros_like(xxx[:,:,0])
    
            # Extract each variable from tecplot as an array
            var = rect_zone.values(name).as_numpy_array().reshape(xxx.shape)
    
            # At each x/y, find the z coord of max beta and save that
            for idy in range(len(yyy[:,0,0])):
                for idx in range(len(xxx[0,:,0])):
                    # If any of the beta values in this column are -1 near the middle, that means we are at the inner edge of the bounding box
                    # In this case, set the data value to 0 to let me know where the boundary is!
                    if np.min(beta_meshgrid[idy,idx,len(z_axis)//4:-len(z_axis)//4]) == -1:
                        data[name][idy,idx] = 0
                    else:
                        # Find the indices of the two nearest points
                        lower_idz = np.searchsorted(zzz[idy,idx,:], data['Z'][idy,idx]) - 1
                        upper_idz = lower_idz + 1
    
                        # Get the coordinates of the nearest points
                        Z_lower = zzz[idy,idx,lower_idz]
                        Z_upper = zzz[idy,idx,upper_idz]
                        var_lower = var[idy,idx,lower_idz]
                        var_upper = var[idy,idx,upper_idz]
                        
                        # Perform linear interpolation
                        data[name][idy,idx] = var_lower + (var_upper - var_lower) * (data['Z'][idy,idx] - Z_lower) / (Z_upper - Z_lower)
    
        print("Done!")
        save_file = open(str(dir+file[:-4]+"MHDTEST_csdata_t_"+'{:06.2f}'.format(round(time,2))), 'wb') 
        pickle.dump(data, save_file) 


def smooth_meshgrid(X, Y, Z, smoothing_param):
    """
    Smooth the Z values of a meshgrid defined by X, Y coordinates using a Gaussian filter.
    
    Parameters:
    X (2D numpy array): The X coordinates of the meshgrid.
    Y (2D numpy array): The Y coordinates of the meshgrid.
    Z (2D numpy array): The Z coordinates of the meshgrid.
    smoothing_param (float): The standard deviation for the Gaussian kernel, controlling the smoothing.
    
    Returns:
    Z_smoothed (2D numpy array): The smoothed Z values of the meshgrid.
    """
    # Check if X, Y, Z are of the same shape
    if X.shape != Y.shape or X.shape != Z.shape:
        raise ValueError("X, Y, and Z meshgrids must have the same shape")
    
    # Apply Gaussian filter to the Z meshgrid
    Z_smoothed = gaussian_filter(Z, sigma=smoothing_param)
    
    return Z_smoothed

def plot_sphere(ax, radius=1, center=(0, 0, 0), color='b', alpha=0.5, zorder = 1,quarter=False, xlims = [-10,10], ylims = [-10,10], zlims = [-10,10]):
    """
    Plots a sphere of given radius centered at center on the provided 3D axis.
    
    Parameters:
    - ax: The 3D axis to plot the sphere on.
    - radius: The radius of the sphere (default: 1).
    - center: The (x, y, z) coordinates of the sphere's center (default: (0, 0, 0)).
    - color: The color of the sphere (default: blue).
    - alpha: The transparency of the sphere (default: 0.5).
    """
    u = np.linspace(np.pi/2, 3/2 * np.pi, 100)
    if quarter:
        v = np.linspace(0, np.pi/2, 100)
    else:
        v = np.linspace(0, np.pi, 100)
    x = radius * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = radius * np.outer(np.sin(u), np.sin(v)) + center[1]
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]

    # Mask out any values outside the axes lims
    mask = (x < xlims[0]) | (x > xlims[1]) | (y < ylims[0]) | (y > ylims[1]) | (z < zlims[0]) | (z > zlims[1]) 
    x[mask] = np.nan
    y[mask] = np.nan
    z[mask] = np.nan

    ax.plot_surface(x, y, z, color=color, alpha=alpha, zorder=zorder)

def average_value(var_ls,t0,t_start,t_stop,type='csdata'):
    # Input: Variables to average, the current time (t0), and the times relative to present to average over (t0+t_start to t0+t_stop)
    # Ouput: dictionary of arrays of time-averaged values

    averages = {}
    count = 0

    temp_files = get_files(dir,key="3d\_fluid.*"+type+"\_t\_...\...",read_time = True, reduce = False)

    for t in list(temp_files.keys()): 
        # Check to see if this file is in the time range we want
        if (float(t) >= (t0+t_start)) and (float(t) <= t0+t_stop):
            temp_file = str(temp_files[t])
    
            # Read in this data
            with open(dir+temp_file, 'rb') as f:
                temp_data = pickle.load(f) 
            
            # Add the data to our running average for each variable
            for var in var_ls:
                if var not in averages.keys():
                    averages[var] = temp_data[var]
                else:
                    averages[var] += temp_data[var]

            count += 1
        
    # Divide by total time steps
    for var in var_ls:
        averages[var] = averages[var]/count

    return averages

def find_indices(X, Y, XX, YY):
    # Function used to get ix and iy for some coordinates X and Y
    X = np.array(X)
    Y = np.array(Y)
    XX = np.array(XX)
    YY = np.array(YY)

    ix = []
    iy = []
    
    for (x, y) in zip(X, Y):
        # Find the closest index in the meshgrid for the x coordinate
        ix_index = np.abs(XX[0] - x).argmin()
        # Find the closest index in the meshgrid for the y coordinate
        iy_index = np.abs(YY[:, 0] - y).argmin()
        
        ix.append(ix_index)
        iy.append(iy_index)
        
    return iy, ix

def remove_duplicate_rows(arr):
    # Used in df_tracker... does something to remove repeated rows in the matching matrix
    seen = set()
    filtered_rows = []
    for row in arr:
        if row[0] not in seen:
            filtered_rows.append(row)
            seen.add(row[0])
    return np.array(filtered_rows)

def find_boundary_points(X, Y):
    # Combine the coordinate lists into a single array of points
    points = np.column_stack((X, Y))

    # Compute the convex hull of the points
    hull = ConvexHull(points)

    # Extract the boundary points
    boundary_points = hull.vertices

    # Boundary points in original coordinate lists
    boundary_X = points[boundary_points, 0]
    boundary_Y = points[boundary_points, 1]

    return boundary_X.tolist(), boundary_Y.tolist()

def create_above_surface_mask(X, Y, Z, XX, YY, ZZ):
    # Works out all the 3D points above a 2D surface ie all the points above the current sheet.
    # Used for 3D plotting to determine what is above what.
    # Check that XX, YY, ZZ have the same shape
    assert XX.shape == YY.shape == ZZ.shape, "Arrays XX, YY, and ZZ must have the same shape"
    
    # Check that X, Y, Z have the same shape
    assert X.shape == Y.shape == Z.shape, "Arrays X, Y, and Z must have the same shape"

    # Determine the shape of the input arrays
    nx, ny, nz = XX.shape

    # Initialize a mask with the same shape as ZZ
    mask = np.zeros_like(ZZ, dtype=bool)

    # Iterate over the entire 3D meshgrid
    for i in range(nx):
        for j in range(ny):
            # Find the index in the 2D arrays corresponding to the x and y coordinates
            xi = np.argmin(np.abs(X[0] - XX[i, j, 0]))
            yi = np.argmin(np.abs(Y[:, 0] - YY[i, j, 0]))

            # Compare ZZ with Z to determine the mask
            mask[i, j, :] = ZZ[i, j, :] > Z[yi, xi]

    return mask


def compute_dt(var_ls,time,type='csdata'):
    # Input: variables to compute the time derivative for, and the current time
    # Output: dictionary of time derivatives for each variable, calculated as dvar_dt = var(time+dt)-var(time-dt)/(2*dt)
    # If earlier or later times are not available, we do either var(time+dt)-var(time)/dt or var(time)-var(time-dt)/dt

    # Declare output dictionary
    deriv_dict = {}
    #print("Computing time derivatives at time",time)
    key_minus = '{:.2f}'.format((float(time)-dt), 'wb')
    key_plus = '{:.2f}'.format((float(time)+dt), 'wb')
    #print(files.keys())
    #print(key_minus)
    #print(key_plus)

    if type=='csdata':
        temp_files = filescs
        data = datacs
    elif type=='numpy':
        temp_files = files3D
        data = data3d

    # Read in the data depending on whether its available
    if (key_minus in temp_files.keys()) and (key_plus in temp_files.keys()):
        # Case one: earlier and later timestep available, so use both
        #print("Earlier and later timesteps available!")
        with open(dir+temp_files[key_minus], 'rb') as f:
            data_tminus = pickle.load(f) 
        with open(dir+temp_files[key_plus], 'rb') as f:
            data_tplus = pickle.load(f) 
        for var in var_ls:
            deriv_dict[var] = (data_tplus[var]-data_tminus[var])/(2*dt)
    
    elif (key_minus in temp_files.keys()):
        # Case two: only earlier time available
        #print("Only earlier timestep available!")
        with open(dir+temp_files[key_minus], 'rb') as f:
            data_tminus = pickle.load(f) 
        for var in var_ls:
            deriv_dict[var] = (data[var]-data_tminus[var])/(dt)

    elif (key_plus in temp_files.keys()):
        # Case three: only later time available
        #print("Only later timestep available!")
        print(dir+temp_files[key_plus])
        with open(dir+temp_files[key_plus], 'rb') as f:
            data_tplus = pickle.load(f) 
        for var in var_ls:
            deriv_dict[var] = (data_tplus[var]-data[var])/(dt)

    else:
        print("ERROR: NO OTHER TIMESTEPS FOUND FOR DERIVATIVE AT TIME =",time)
            
    return deriv_dict

def plane_intersection(x, y, z, plane_z=0.2):
    # Used in 3D_df_tracker2 to check whether a given field line intersects the current sheet multiple times or not
    intersections = 0  # Counter for intersections with the plane

    # Iterate over the list of points
    for i in range(1, len(z)):
        if (z[i-1] - plane_z) * (z[i] - plane_z) < 0:
            # There is an intersection between z[i-1] and z[i] since their signs are different
            intersections += 1
        elif z[i-1] == plane_z and z[i] != plane_z:
            # Edge case: the point is exactly on the plane
            intersections += 1

    return intersections

def symlog(x):
    """ Returns the symmetric log10 value """
    return np.sign(x) * np.log10(np.abs(x))

def symroot(x):
    """ Returns the symmetric sqrt value """
    return np.sign(x) * np.sqrt(np.abs(x))

# Functions for adding arrows; use "setattr(Axes3D, 'arrow3D', _arrow3D)" after defining your axes
class Arrow3D(FancyArrowPatch):
            def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
                super().__init__((0, 0), (0, 0), *args, **kwargs)
                self._xyz = (x, y, z)
                self._dxdydz = (dx, dy, dz)
        
            def draw(self, renderer):
                x1, y1, z1 = self._xyz
                dx, dy, dz = self._dxdydz
                x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)
        
                xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
                self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
                super().draw(renderer)
                
            def do_3d_projection(self, renderer=None):
                x1, y1, z1 = self._xyz
                dx, dy, dz = self._dxdydz
                x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)
        
                xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
                self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
                
                return np.min(zs) 
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

In [ ]:
############################## PREPROCESSING ##############################
dir = "/Users/atcushen/Documents/MercuryModelling/runs/nightside_v4_run1/ta-2e/"
pre_proc_MHD = True
MHD_start_time = 40  # First time step of MHD data saved in this directory
t_bound = [40,42]    # Start and stop times of this data to be plot (cannot go beyond available PIC data time range)
MHD_dt = 1 # File time cadence of MHD data [s]
preproc_xlims = [-10,4]
preproc_ylims = [-5,5]
preproc_zlims = [-5,5]
MHD_var_ls = ["B_x [[]nT[]]","B_y [[]nT[]]","B_z [[]nT[]]"]
            #,"Ex","Ey","Ez","rhoS0","uxS0","uyS0","uzS0","pxxS0","pyyS0","pzzS0","pxyS0","pxzS0","pyzS0",
          #"rhoS1","uxS1","uyS1","uzS1","pxxS1","pyyS1","pzzS1","pxyS1","pxzS1","pyzS1","Jx","Jy","Jz","dp_dx","dp_dy","dp_dz",
          #"dB_dx","dB_dy","dB_dz","duix_dx","duiy_dx","duiz_dx","duix_dy","duiy_dy","duiz_dy","duix_dz","duiy_dz","duiz_dz"]

# Cell data from PIC (found by directly examining the actual output grid)
PIC_xmin = -3.99219
PIC_ymin = -1.22656
PIC_zmin = -0.742188
PIC_dx = 0.01562501
dx_factor = 20 # The MHD grid will be limited to steps of this size factor 
PIC_dx1 = PIC_dx * dx_factor

############################## PREPROCESSING ##############################

if pre_proc_MHD:
    xmin_idx = int(round((PIC_xmin - preproc_xlims[0])/PIC_dx1,0)) # How many PIC_dx between PIC_xmin and preproc_xlims[0]
    ymin_idx = int(round((PIC_ymin - preproc_ylims[0])/PIC_dx1,0))
    zmin_idx = int(round((PIC_zmin - preproc_zlims[0])/PIC_dx1,0))
    xmax_idx = int(round((preproc_xlims[1] - PIC_xmin)/PIC_dx1,0))
    ymax_idx = int(round((preproc_ylims[1] - PIC_ymin)/PIC_dx1,0))
    zmax_idx = int(round((preproc_zlims[1] - PIC_zmin)/PIC_dx1,0))

    MHD_xlims = [PIC_xmin - xmin_idx*PIC_dx1, PIC_xmin + xmax_idx*PIC_dx1]
    MHD_ylims = [PIC_ymin - ymin_idx*PIC_dx1, PIC_ymin + ymax_idx*PIC_dx1]
    MHD_zlims = [PIC_zmin - zmin_idx*PIC_dx1, PIC_zmin + zmax_idx*PIC_dx1]

    MHD_xaxis = np.arange(MHD_xlims[0],MHD_xlims[1]+PIC_dx1,PIC_dx1)
    MHD_yaxis = np.arange(MHD_ylims[0],MHD_ylims[1]+PIC_dx1,PIC_dx1)
    MHD_zaxis = np.arange(MHD_zlims[0],MHD_zlims[1]+PIC_dx1,PIC_dx1)
    
    MHD_files = get_files(dir,MHD_start_time,t_bound,MHD_dt,key="3d\_\_var\_3\_t.*\.plt",reduce = True)
    for time in list(MHD_files.keys()): 
        print("Preprocessing MHD data for t =",time)
        file = str(MHD_files[time])
    
        # Read in dataset
        dataset = read_dataset(dir+file,port=7605)
        # Save .plt as numpy data
        data = MHD_to_numpy(dataset,MHD_xaxis,MHD_yaxis,MHD_zaxis,var_ls=MHD_var_ls)
    

Preprocessing MHD data for t = 40
reading: /Users/atcushen/Documents/MercuryModelling/runs/nightside_v4_run1/ta-2e/3d__var_3_t00000040_n00313709.plt
Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7605
Connection established.


In [134]:
dataset = read_dataset(dir+file,port=7605)
x_axis = MHD_xaxis
y_axis = MHD_yaxis
z_axis = MHD_zaxis
rect_zone = dataset.add_ordered_zone('rect_zone',[len(x_axis),len(y_axis),len(z_axis)])

# Create 3D coordinate meshgrids
xxx,yyy,zzz = np.meshgrid(x_axis,y_axis,z_axis)

# Assign coordinate values to the rect_zone using the meshgrids
rect_zone.values('X [R]')[:] = xxx.ravel()
rect_zone.values('Y [R]')[:] = yyy.ravel()
rect_zone.values('Z [R]')[:] = zzz.ravel()

reading: /Users/atcushen/Documents/MercuryModelling/runs/nightside_v4_run1/ta-2e/3d__var_3_t00000040_n00313709.plt
Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7605
Connection established.


AttributeError: 'NoneType' object has no attribute 'index'

In [142]:
rect_zone.values('X [[]R[]]')

In [114]:
MHD_xaxis

array([-9.99219384e+00, -9.97656883e+00, -9.96094382e+00, -9.94531881e+00,
       -9.92969380e+00, -9.91406879e+00, -9.89844378e+00, -9.88281877e+00,
       -9.86719376e+00, -9.85156875e+00, -9.83594374e+00, -9.82031873e+00,
       -9.80469372e+00, -9.78906871e+00, -9.77344370e+00, -9.75781869e+00,
       -9.74219368e+00, -9.72656867e+00, -9.71094366e+00, -9.69531865e+00,
       -9.67969364e+00, -9.66406863e+00, -9.64844362e+00, -9.63281861e+00,
       -9.61719360e+00, -9.60156859e+00, -9.58594358e+00, -9.57031857e+00,
       -9.55469356e+00, -9.53906855e+00, -9.52344354e+00, -9.50781853e+00,
       -9.49219352e+00, -9.47656851e+00, -9.46094350e+00, -9.44531849e+00,
       -9.42969348e+00, -9.41406847e+00, -9.39844346e+00, -9.38281845e+00,
       -9.36719344e+00, -9.35156843e+00, -9.33594342e+00, -9.32031841e+00,
       -9.30469340e+00, -9.28906839e+00, -9.27344338e+00, -9.25781837e+00,
       -9.24219336e+00, -9.22656835e+00, -9.21094334e+00, -9.19531833e+00,
       -9.17969332e+00, -

In [51]:
############################## USER INPUT ##############################

# Plotting control parameters
# Directory data
dir = "/Users/atcushen/Documents/MercuryModelling/runs/nightside_v4_run1/ta-234e/"   # Directory with data 

# Time range: this sets the times and timing of the plots
t_bound = [40,40.2]    # Start and stop times of this data to be plot (cannot go beyond available PIC data time range)
PIC_start_time = 30  # First time step of PIC data saved in this directory
MHD_start_time = 40  # First time step of MHD data saved in this directory
PIC_dt = 0.05  # File time cadence of PIC data [s]
MHD_dt = 1 # File time cadence of MHD data [s]
cell_size = R_M/64  # Edge length of Pic cell, in m

# Spatial range: this sets the spatial extent of the data to be loaded in
xlims = [-5,0]
ylims = [-2,2]
zlims = [-2,2]

############################## USER INPUT ##############################

In [53]:
# Begin plotting
# Get all the files in the time range
PIC_files_3D = get_files(dir,PIC_start_time,t_bound,PIC_dt,key="3d\_fluid.*numpy\_t\_...\...",read_time = True,reduce = True)
PIC_files_cs = get_files(dir,PIC_start_time,t_bound,PIC_dt,key="3d\_fluid.*csdata\_t\_...\...",read_time = True,reduce = True)
MHD_files = get_files(dir,MHD_start_time,t_bound,MHD_dt,key="3d\_\_var\_3\_t.*\.plt",reduce = True)

# Begin iteration loop
iter = 0
for time_str in list(PIC_files_3D.keys()): 
    time = float(time_str)
    print("Plotting t =",time)

    # Read in the 3D PIC data
    my_file = str(PIC_files_3D[time_str])
    with open(dir+my_file, 'rb') as f:
        PIC_data_3D = pickle.load(f) 

    # Check if we need to load in MHD data to fill gaps (usually is the case)
    if np.min(PIC_data_3D['X']) > xlims[0] or np.min(PIC_data_3D['X']) < xlims[0] or np.min(PIC_data_3D['Y']) > ylims[0] or np.max(PIC_data_3D['Y']) < ylims[1] or np.min(PIC_data_3D['Z']) > zlims[0] or np.max(PIC_data_3D['Z']) < zlims[1]:
        print("!")

Plotting t = 40.0
!
Plotting t = 40.05
!
Plotting t = 40.1
!
Plotting t = 40.15
!


In [68]:
PIC_data_3D['Z'][0,0,0]

-0.742188

In [85]:
ls = []
for i in range(1,len(PIC_data_3D['X'][0,:,0])):
    #print(i,"  :  ",PIC_data_3D['X'][0,i,0] - PIC_data_3D['X'][0,i-1,0])
    ls.append(PIC_data_3D['X'][0,i,0] - PIC_data_3D['X'][0,i-1,0])
print(np.mean(ls))

0.01562501


In [74]:
i=5
PIC_data_3D['X'][0,i,0] - PIC_data_3D['X'][0,i-1,0]

0.015629768

In [76]:
val

-6.127450980392157e-05